In [15]:
import pandas as pd
from logs_parser.parser import parse_mjlog
from logs_parser.viewer import print_node
import xml.etree.ElementTree as ET
import preprocess
import sys
import json

df = pd.read_csv('./logs_parser/2021.csv')
for i, xml_str in enumerate(df['log_content']):
    node = ET.fromstring(xml_str)
    data = parse_mjlog(node)
    if len(data["meta"]['UN'][3]["name"]) == 0:
        continue

    if i == 2:
        f = open('example.json','w')
        f.write(json.dumps(data, indent=2))
        f.close()
        sys.exit()
    res = {}
    for i in range(len(data["rounds"])):

        round_data = data["rounds"][i] 
        round_info = preprocess.get_round_info(round_data) 


SystemExit: 

In [16]:
from mahjong.hand_calculating.hand import HandCalculator
from mahjong.tile import TilesConverter
from mahjong.hand_calculating.hand_config import HandConfig
from mahjong.meld import Meld
from mahjong.shanten import Shanten


def getFeatures(round_info):
    features = {}
    

    def shanten(tiles_list):
        #given 14
        tiles = TilesConverter.to_34_array(tiles_list)
        return Shanten.calculate_shanten(tiles)

    def doraAround(round_info):
        dora_indicators = np.array(round_info['dora_indicators']) // 4
        res = 0
        for t in tiles:
            t //= 4
            for d in dora_indicators:
                if abs(d-t) <= 2:
                    res += 1
                    break
        return res
    
    
    


# def shanten1(tiles_list):
#     tiles = TilesConverter.to_34_array(tiles_list)
#     shanten_results = Shanten.calculate_shanten_for_regular_hand(tiles)
#     shanten_results.append(Shanten.calculate_shanten_for_chiitoitsu_hand(tiles))
#     shanten_results.append(Shanten.calculate_shanten_for_kokushi_hand)
    

    

In [33]:
from typing import List
from utils import *

def is_tile_strictly_isolated(hand_34, tile_34):
    """
    Tile is strictly isolated if it doesn't have -2, -1, 0, +1, +2 neighbors
    :param hand_34: array of tiles in 34 tile format
    :param tile_34: int
    :return: bool
    """

    if is_honor(tile_34):
        return hand_34[tile_34] - 1 <= 0

    simplified = simplify(tile_34)

    # 1 suit tile
    if simplified == 0:
        indices = [tile_34, tile_34 + 1, tile_34 + 2]
    # 2 suit tile
    elif simplified == 1:
        indices = [tile_34 - 1, tile_34, tile_34 + 1, tile_34 + 2]
    # 8 suit tile
    elif simplified == 7:
        indices = [tile_34 - 2, tile_34 - 1, tile_34, tile_34 + 1]
    # 9 suit tile
    elif simplified == 8:
        indices = [tile_34 - 2, tile_34 - 1, tile_34]
    # 3-7 tiles tiles
    else:
        indices = [tile_34 - 2, tile_34 - 1, tile_34, tile_34 + 1, tile_34 + 2]

    isolated = True
    for tile_index in indices:
        # we don't want to count our tile as it is in hand already
        if tile_index == tile_34:
            isolated &= hand_34[tile_index] - 1 <= 0
        else:
            isolated &= hand_34[tile_index] == 0

    return isolated



def calculate_waits(closed_hand_34: List[int], all_tiles_34: List[int], use_chiitoitsu: bool = True):
    shanten = Shanten()
    previous_shanten = shanten.calculate_shanten(closed_hand_34, use_chiitoitsu=use_chiitoitsu)

    waiting = []
    for tile_index in range(0, 34):
        # it is important to check that we don't have all 4 tiles in the all tiles
        # not in the closed hand
        # so, we will not count 1z as waiting when we have 111z meld
        if all_tiles_34[tile_index] == 4:
            continue

        closed_hand_34[tile_index] += 1

        skip_isolated_tile = True
        if closed_hand_34[tile_index] == 4:
            skip_isolated_tile = False
        if use_chiitoitsu and closed_hand_34[tile_index] == 3:
            skip_isolated_tile = False

        # there is no need to check single isolated tile
        if skip_isolated_tile and is_tile_strictly_isolated(closed_hand_34, tile_index):
            closed_hand_34[tile_index] -= 1
            continue

        new_shanten = shanten.calculate_shanten(closed_hand_34, use_chiitoitsu=use_chiitoitsu)

        if new_shanten == previous_shanten - 1:
            waiting.append(tile_index)

        closed_hand_34[tile_index] -= 1

    return waiting, previous_shanten

closed_hand = TilesConverter.string_to_34_array(man='12334555677889')
print(closed_hand)
shanten = Shanten()
print(shanten.calculate_shanten(closed_hand))
calculate_waits(closed_hand, closed_hand)

[1, 1, 2, 1, 3, 1, 2, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
-1


AssertionError: Too many tiles = 15